# Getting Started with Polaris and Snowflake and Spark | hands on Guide

# Create an Account on snowflake 

```
CREATE ACCOUNT MYACCOUNT
   ADMIN_NAME = ''
   ADMIN_PASSWORD = ''
   MUST_CHANGE_PASSWORD = FALSE
   EMAIL = ''
   EDITION = STANDARD
   REGION = 'AWS_US_EAST_1'
   POLARIS = TRUE;
```


#### Create Iam Role and policy
```
chmod +x ./role.sh
./role.sh
```


# Step 1: Create Spark Session 

In [5]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"

import pyspark
from pyspark.sql import SparkSession
import os

# Set environment variables for sensitive information
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"
os.environ["ICEBERG_ACCESS_DELEGATION"] = "vended-credentials"
os.environ["ICEBERG_URI"] = "https://XXXXXXXX.snowflakecomputing.com/polaris/api/catalog"
os.environ["ICEBERG_CREDENTIAL"] = "XXX:XXXXX="
os.environ["ICEBERG_WAREHOUSE"] = "XXX"
os.environ["ICEBERG_SCOPE"] = "PRINCIPAL_ROLE:XXXX"


# Initialize Spark session with the sensitive items sourced from environment variables
spark = SparkSession.builder.appName('iceberg_lab') \
.config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.1,software.amazon.awssdk:bundle:2.20.160,software.amazon.awssdk:url-connection-client:2.20.160') \
.config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
.config('spark.sql.defaultCatalog', 'opencatalog') \
.config('spark.sql.catalog.opencatalog', 'org.apache.iceberg.spark.SparkCatalog') \
.config('spark.sql.catalog.opencatalog.type', 'rest') \
.config('spark.sql.catalog.opencatalog.header.X-Iceberg-Access-Delegation', os.environ["ICEBERG_ACCESS_DELEGATION"]) \
.config('spark.sql.catalog.opencatalog.uri', os.environ["ICEBERG_URI"]) \
.config('spark.sql.catalog.opencatalog.credential', os.environ["ICEBERG_CREDENTIAL"]) \
.config('spark.sql.catalog.opencatalog.warehouse', os.environ["ICEBERG_WAREHOUSE"]) \
.config('spark.sql.catalog.opencatalog.scope', os.environ["ICEBERG_SCOPE"]) \
.getOrCreate()

In [6]:
spark

In [9]:
#Show namespaces
spark.sql("show namespaces").show()

#Create namespace
spark.sql("create namespace if not exists spark_demo")

spark.sql("use namespace spark_demo")

+----------+
| namespace|
+----------+
|spark_demo|
+----------+


DataFrame[]

In [10]:
# Create the 'customer' table with sample schema
spark.sql("""
CREATE TABLE IF NOT EXISTS customer (
    customer_id INT,
    first_name STRING,
    last_name STRING,
    email STRING,
    phone STRING,
    address STRING,
    city STRING,
    state STRING,
    zip_code STRING,
    country STRING
) USING iceberg
""")

# Insert some sample mock data into the 'customer' table
spark.sql("""
INSERT INTO customer VALUES
(1, 'John', 'Doe', 'johndoe@example.com', '555-1234', '123 Main St', 'New York', 'NY', '10001', 'USA'),
(2, 'Jane', 'Smith', 'janesmith@example.com', '555-5678', '456 Maple Ave', 'Los Angeles', 'CA', '90001', 'USA'),
(3, 'Sam', 'Brown', 'sambrown@example.com', '555-9876', '789 Oak St', 'Chicago', 'IL', '60601', 'USA'),
(4, 'Emily', 'Jones', 'emilyjones@example.com', '555-4321', '101 Pine St', 'San Francisco', 'CA', '94101', 'USA')
""")



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

# Query Iceberg tables

In [26]:
# Query the 'customer' table to verify the records
spark.sql("SELECT * FROM customer").show()

+-----------+----------+---------+--------------------+--------+-------------+-------------+-----+--------+-------+
|customer_id|first_name|last_name|               email|   phone|      address|         city|state|zip_code|country|
+-----------+----------+---------+--------------------+--------+-------------+-------------+-----+--------+-------+
|          1|      John|      Doe| johndoe@example.com|555-1234|  123 Main St|     New York|   NY|   10001|    USA|
|          2|      Jane|    Smith|janesmith@example...|555-5678|456 Maple Ave|  Los Angeles|   CA|   90001|    USA|
|          3|       Sam|    Brown|sambrown@example.com|555-9876|   789 Oak St|      Chicago|   IL|   60601|    USA|
|          4|     Emily|    Jones|emilyjones@exampl...|555-4321|  101 Pine St|San Francisco|   CA|   94101|    USA|
+-----------+----------+---------+--------------------+--------+-------------+-------------+-----+--------+-------+


 ```

CREATE OR REPLACE EXTERNAL VOLUME polaris_volume
STORAGE_LOCATIONS =
  (
     (
        NAME = 'ext_vol'
        STORAGE_PROVIDER = 'S3'
        STORAGE_BASE_URL = 's3://<BUCKET>/demo-polaris/'
        STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::<ACCOUNT>:role/snowflake_int_role'
        STORAGE_AWS_EXTERNAL_ID = 'ext_vol'
     )
  );

DESC EXTERNAL VOLUME polaris_volume;

  
create or replace iceberg table customer
  catalog = 'demo_open_catalog_int'
  external_volume = 'polaris_volume'
  catalog_table_name = 'customer'

select * from customer;
```